In [ ]:
# import Pkg
# Pkg.add("NBInclude")
# Pkg.add("StatsBase")
# Pkg.add("Manopt")
# Pkg.add("Manifolds")
# Pkg.add("Distances")
# import the simulation code
import NBInclude
import Random
import StatsBase
#import Manopt
import Manifolds
import Distances
using Manopt

In [ ]:
# import the code from the l,d motif simulation notebook
NBInclude.@nbinclude("simulate_ld_motif.ipynb")

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 14, 100, 1)

In [ ]:
# write the scoring function
# it is written in such a way that it is continuously defined
function Score(M, Starts_inp, Seqs=sequences, Length=length(motif))
    maxScore = Length * length(Seqs)
    
    # handle bad inputs
    if Starts_inp isa Vector{Float64} 
        Starts_down = Int.(floor.(Starts_inp))
        Starts_up = Int.(ceil.(Starts_inp))
    else
        Starts_down = [i for i in Starts_inp]
        Starts_up = [i + 1 for i in Starts_inp]
    end

    Starts_up[Starts_up .< 1] .= 1
    Starts_up[Starts_up .> (length(Seqs[1]) - Length + 1)] .= length(Seqs[1]) - Length + 1
    
    Starts_down[Starts_down .< 1] .= 1
    Starts_down[Starts_down .> (length(Seqs[1]) - Length + 1)] .= length(Seqs[1]) - Length + 1

    # extract the sequences to iterate over
    seqsMatrix_down = permutedims(reduce(hcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_down)))
        
    seqsMatrix_up = permutedims(reduce(hcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_up)))

    

    
    # find the most common element in each column
    mostCommon_down = mapslices(StatsBase.mode, seqsMatrix_down, dims = 1)
    mostCommon_up = mapslices(StatsBase.mode, seqsMatrix_up, dims = 1)
    
    # now count how many sequences are equal to the consensus
    thisScore_down = sum(map((i, j) -> sum(i .== j), eachslice(seqsMatrix_down, dims = 2), mostCommon_down))
    thisScore_up = sum(map((i, j) -> sum(i .== j), eachslice(seqsMatrix_up, dims = 2), mostCommon_up))

    
    dist1 = Distances.euclidean([i for i in Starts_inp], Starts_down)
    dist2 = Distances.euclidean([i for i in Starts_inp], Starts_up)
    
    thisScore = (1 - (dist1)/(dist1 + dist2)) * thisScore_down + (1 - (dist2)/(dist1 + dist2)) * thisScore_up        

    if (isnan(thisScore)) | (thisScore > maxScore)
        thisScore = 0
    end
    # figure out the euclidean distance for Starts from Up and Down
    
    return(maxScore - thisScore)
end

In [ ]:
iterTup = [1:(len - length(motif) + 1) for len in length.(sequences)]

In [ ]:
# then use tuple unpacking
allScores = map(i -> Score(0, i), Iterators.product(iterTup...))

In [ ]:
size(allScores)

In [ ]:
findmin(allScores)

In [ ]:
# ok so everything works now
# lets try and use particle swarm!
M = Manifolds.Euclidean(length(sequences))

In [ ]:
# random points
n_ = 100
pts = map(i -> (length(sequences[1]) - length(motif) - 1) * rand(3) .+ 1, 1:n_)

In [ ]:
vecs = map(i -> (5 * (2 * rand(3) .- 1)), 1:n_)

In [ ]:
res =particle_swarm!(M, Score, n = n_, x0 = pts, velocity = vecs, inertia = 0.8, return_options = true)

In [ ]:
res(M,Score)

In [ ]:
typeof(res)

In [ ]:
type(Manopt.particle_swarm(M, Score, n = n_, x0 = pts, velocity = vecs, inertia = 0.8, return_options = true))

In [ ]:
# check how good the answer is
Score(0, res)

In [ ]:
M

In [ ]:
Score

In [ ]:
res

In [ ]:
pts

In [ ]:
get_solver_result(res)